In [1]:
import os
import csv
import os.path as osp

import pandas as pd
import numpy as np

import iFeatureOmegaCLI

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

from sklearn.feature_selection import chi2
from sklearn.metrics import f1_score, precision_score, recall_score

In [2]:
BASE_DIR = os.getcwd()

DATA_DIR = osp.join(BASE_DIR, 'Data')

In [3]:
SEED = 42
np.random.seed(SEED)

In [4]:
#positional concatenation of mirnaseq with mirna_esa
def with_esa_mirna(miRNA, miRNA_esa):
    miRNA_withESA=[]
    for i in range(len(miRNA)):
        con_mirna_esa = miRNA_esa[i][:10] + miRNA[i][10:]
        con_mirna_esa = miRNA_esa[i] + miRNA[i][len(miRNA_esa[i]):]
        miRNA_withESA.append(con_mirna_esa)
    
    return miRNA_withESA

In [5]:
def with_esa_mrna(rev_mRNA,mRNA_CTS_esa):
    mRNA_withESA=[]
    for i in range(len(rev_mRNA)):
        con_mrna_esa = rev_mRNA[i][:5] + mRNA_CTS_esa[i] + rev_mRNA[i][len(rev_mRNA[i][:5] + mRNA_CTS_esa[i]):]
        mRNA_withESA.append(con_mrna_esa)
    return mRNA_withESA

In [6]:
def get_sequences(df):
    mrna = with_esa_mrna(df.rev_mRNA, df.mRNA_CTS_esa)
    mirna = with_esa_mirna(df.miRNA, df.miRNA_esa)
    labels = df.Label.tolist()
    return mrna, mirna, labels

In [7]:
def get_mirna_sequence(mirna_list, filename):
    with open(filename, "w") as f:
        for i, seq in enumerate(mirna_list):
            line = f">seq_{i}\n{seq}\n"
            f.write(line)
    return None

In [8]:
def get_mrna_sequence(mrna_list, filename):
    with open(filename, "w") as f:
        for i, seq in enumerate(mrna_list):
            line = f">seq_{i}\n{seq}\n"
            f.write(line)
    return None

In [9]:
def get_features(filepath):
    df = pd.read_csv(filepath)
    base_path = "/".join(filepath.split("/")[:-1])
    fname_mi = f'{filepath.split("/")[-1].split(".")[0]}_features_mirna.txt'
    fname_mr = f'{filepath.split("/")[-1].split(".")[0]}_features_mrna.txt'
    new_path_mi = osp.join(base_path, fname_mi)
    new_path_mr = osp.join(base_path, fname_mr)
    try:
        mrna_list, mirna_list, labels = get_sequences(df)
        mirna_seq = get_mirna_sequence(mirna_list, new_path_mi)
        mrna_seq = get_mrna_sequence(mrna_list, new_path_mr)
        
#         mi_feature_scp = iFeatureOmegaCLI.iRNA(new_path_mi)
#         mi_feature_scp.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
#         mi_feature_scp.get_descriptor(descriptor= 'SCPseDNC')
        
#         mr_feature_scp = iFeatureOmegaCLI.iRNA(new_path_mr)
#         mr_feature_scp.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
#         mr_feature_scp.get_descriptor(descriptor= 'SCPseDNC')
        
#         X1 = np.concatenate((mi_feature_scp.encodings.values, mr_feature_scp.encodings.values),axis=1)

#         mi_feature_pcp = iFeatureOmegaCLI.iRNA(new_path_mi)
#         mi_feature_pcp.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
#         mi_feature_pcp.get_descriptor(descriptor= 'PCPseDNC')
        
#         mr_feature_pcp = iFeatureOmegaCLI.iRNA(new_path_mr)
#         mr_feature_pcp.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
#         mr_feature_pcp.get_descriptor(descriptor= 'PCPseDNC')
        
#         X2 = np.concatenate((mi_feature_pcp.encodings.values, mr_feature_pcp.encodings.values),axis=1)
        
#         mi_feature_kmer = iFeatureOmegaCLI.iRNA(new_path_mi)
#         mi_feature_kmer.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
#         mi_feature_kmer.get_descriptor(descriptor= 'Kmer type 1')
        
#         mr_feature_kmer = iFeatureOmegaCLI.iRNA(new_path_mr)
#         mr_feature_kmer.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
#         mr_feature_kmer.get_descriptor(descriptor= 'Kmer type 1')
        
#         X = np.concatenate((mi_feature_kmer.encodings.values, mr_feature_kmer.encodings.values),axis=1)
        
#         mi_feature_dpcp = iFeatureOmegaCLI.iRNA(new_path_mi)
#         mi_feature_dpcp.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
#         mi_feature_dpcp.get_descriptor(descriptor= 'DPCP')
        
#         mr_feature_dpcp = iFeatureOmegaCLI.iRNA(new_path_mr)
#         mr_feature_dpcp.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
#         mr_feature_dpcp.get_descriptor(descriptor= 'DPCP')
        
#         X = np.concatenate((mi_feature_dpcp.encodings.values, mr_feature_dpcp.encodings.values),axis=1)
        
        mi_feature_psknc = iFeatureOmegaCLI.iRNA(new_path_mi)
        mi_feature_psknc.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
        mi_feature_psknc.get_descriptor(descriptor= 'PseKNC')
        
        mr_feature_psknc = iFeatureOmegaCLI.iRNA(new_path_mr)
        mr_feature_psknc.import_parameters('/home/dhakal/TPiFO/Parameters/parameters.json')
        mr_feature_psknc.get_descriptor(descriptor= 'PseKNC')
        
        X = np.concatenate((mi_feature_psknc.encodings.values, mr_feature_psknc.encodings.values),axis=1)

#         X = np.concatenate((X1, X2, X3, X4, X5),axis=1)
        y = np.array(labels, dtype="float32")
        return X, y
    except Exception as e:
        print(e)

In [10]:
# Create features from the Train_Val file
import warnings
warnings.filterwarnings("ignore")
scores = []
for i, (path, directory, files) in enumerate(os.walk(DATA_DIR)):    
    if "miRAW_Train_Validation" in path:
        for file in files:
            if "features" not in file:
                fullpath = osp.join(path, file)
                print(fullpath)
                data = get_features(fullpath)
                X, y = data
                skf = StratifiedKFold(n_splits=4, random_state=SEED, shuffle=True)
                for i, (train_index, val_index) in enumerate(skf.split(X, y)):
                    X_train, y_train = X[train_index], y[train_index]
                    X_val, y_val = X[val_index], y[val_index]                   

/home/dhakal/TPiFO/TPiFO_new/Data/miRAW_Train_Validation/miRAW_Train_Validation.csv
File imported successfully.
File imported successfully.


In [11]:
X_train_original = X_train.copy()
X_val_original = X_val.copy()

In [12]:
X_train_df = pd.DataFrame(X_train_original, index= None, columns=None )
X_val_df = pd.DataFrame(X_val_original, index= None, columns=None )

In [13]:
X_train_df.head()

,0,1,2,3,4,5,6,7,8,9,...,124,125,126,127,128,129,130,131,132,133
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.020778,0.020778,0.041557,0.020778,0.020778,0.000000,0.088266,0.093996,0.090498
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.023295,0.000000,0.023295,0.046590,0.000000,0.046590,0.069885,0.068775,0.054078,0.061822
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.020750,0.041500,0.062249,0.020750,0.041500,0.000000,0.115593,0.067549,0.090614
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.021127,0.000000,0.042253,0.021127,0.042253,0.094325,0.090691,0.075554
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.022088,0.000000,0.000000,0.092647,0.076033,0.058230


In [14]:
X_train_df.shape, X_val_df.shape

((39830, 134), (13276, 134))

In [15]:
# find and remove correlated features
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

corr_features = correlation(X_train_df, 0.95)
print('correlated features: ', len(set(corr_features)) )

correlated features:  0


In [16]:
# removed correlated  features
X_train_df.drop(labels=corr_features, axis=1, inplace=True)
X_val_df.drop(labels=corr_features, axis=1, inplace=True)

X_train_df.shape, X_val_df.shape

((39830, 134), (13276, 134))

In [17]:
X_train = np.array(X_train_df)
X_val = np.array(X_val_df)

X_train.shape, X_val.shape

((39830, 134), (13276, 134))

In [18]:
estimators = [
        ('rf', RandomForestClassifier(n_estimators=379, max_depth=31, random_state=SEED)),
        ('svr', make_pipeline(StandardScaler(),
                              LinearSVC(random_state=SEED))),
#         ('dtc', DecisionTreeClassifier(random_state=SEED)),
        ('xgb', xgb.XGBClassifier(objective="binary:logistic", random_state=SEED)),
        #     ('knn', KNeighborsClassifier(n_neighbors=7)),
        ('svcl', make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=3,p=1))),
        ('svcr', make_pipeline(StandardScaler(), SVC(random_state=SEED, kernel='rbf'))),
        #     ('gnv', GaussianNB())
        ]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
clf_score = round(accuracy_score(y_pred,y_val)*100,4)
scores.append(clf_score)

print("Accuracy score by LR model: "+str(round(sum(scores)/len(scores),4))+" %") 

Accuracy score by LR model: 95.4354 %


In [19]:
# # Train the model (model.fit())

for i, (path, directory, files) in enumerate(os.walk(DATA_DIR)):
    if "Test" in path:
        fname = path.split("/")[-1]
        results_file = osp.join(path, f"{fname}_results_PseKNC.txt")
        print(results_file)
        with open(results_file, "w") as f:
            f.write("mean_score,max_score,label\n")
            for file in files:
                if "features" not in file:
                    try:
                        fullpath = osp.join(path, file)
                        data = get_features(fullpath)
                        X_test, y_test = data
                        X_test_df = pd.DataFrame(X_test, index=None, columns=None)
                        X_test_df.drop(labels=corr_features, axis=1, inplace=True)
                        X_test = np.array(X_test_df)
                        y_pred = clf.predict(X_test)
                        mean_score = float(np.mean(y_pred))
                        max_score = float(np.max(y_pred))
                        label = float(y_test[0])
                        line = f"{mean_score},{max_score},{label}\n"
                        f.write(line)
                    except Exception as e:
                        print(e)
                        print(fullpath)

/home/dhakal/TPiFO/TPiFO_new/Data/miRAW_Test4/miRAW_Test4_results_PseKNC.txt
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
''
cannot unpack non-iterable NoneType object
/home/dhakal/TPiFO/TPiFO_new/Data/miRAW_Test4/seq373.csv
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imp

File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported succes

File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported succes

File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported succes

File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported succes

...

File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported succes

File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
''
cannot unpack non

File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported succes

File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
''
cannot unpack non-iterable NoneType object
/home/dhakal/TPiFO/TPiFO_new/Data/miRAW_Test0/seq410.csv
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
F

File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
''
cannot unpack non-iterable NoneType object
/home/dhakal/TPiFO/TPiFO_new/Data/miRAW_Test0/seq102.csv
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
File imported successfully.
F

In [20]:
acc = []
for i in range(10):
    result_path = osp.join(DATA_DIR,f'miRAW_Test{i}/miRAW_Test{i}_results_PseKNC.txt')
    result_file = pd.read_csv(result_path)
    accuracy = accuracy_score(result_file.label, result_file.max_score)
    f1_scores = f1_score(result_file.label, result_file.max_score)
    pre_score = precision_score(result_file.label, result_file.max_score)
    re_score = recall_score(result_file.label, result_file.max_score)
    print(f"Accuracy of test{i}",accuracy)
    acc.append(accuracy)
print('F1 Score',np.average(f1_scores))
print('Precision Score',np.average(pre_score))
print('Recall Score',np.average(re_score))    
print("Average Accuracy",np.average(acc))

Accuracy of test0 0.7887453874538746
Accuracy of test1 0.7963302752293578
Accuracy of test2 0.7994480220791168
Accuracy of test3 0.7944700460829494
Accuracy of test4 0.7965009208103131
Accuracy of test5 0.7943067033976124
Accuracy of test6 0.7919340054995417
Accuracy of test7 0.7909760589318601
Accuracy of test8 0.7959558823529411
Accuracy of test9 0.7972477064220184
F1 Score 0.826121164437451
Precision Score 0.7221458046767538
Recall Score 0.9650735294117647
Average Accuracy 0.7945915008259585
